In [ ]:
import pandas as pd
from pathlib import Path

def load_gfz(filepath: str) -> pd.DataFrame:
    """Read raw GFZ file (no headers) and explode it to 3-hour resolution."""
    cols = (
        ['year', 'month', 'day', 'julian', 'dec_jul', 'carr_rot', 'bartels'] +
        [f'Kp_{i}' for i in range(8)] +
        [f'ap_{i}' for i in range(8)] +
        ['Ap', 'SN', 'F10_7_obs', 'F10_7_adj', 'qual']
    )
    df = pd.read_csv(filepath, sep=r'\s+', names=cols, engine='python')

    # stack the 8 three-hour columns into long form
    long_frames = []
    for i in range(8):
        tmp = df[['year', 'month', 'day',
                  f'Kp_{i}', f'ap_{i}',
                  'Ap', 'SN', 'F10_7_obs', 'F10_7_adj']].copy()
        tmp['hour'] = i * 3
        tmp.rename(columns={f'Kp_{i}': 'Kp', f'ap_{i}': 'ap'}, inplace=True)
        long_frames.append(tmp)
    gfz = pd.concat(long_frames, ignore_index=True)
    gfz['datetime'] = (
        pd.to_datetime(gfz[['year', 'month', 'day']]) +
        pd.to_timedelta(gfz['hour'], unit='h')
    )
    return gfz[['datetime', 'Kp', 'ap', 'Ap', 'SN', 'F10_7_obs', 'F10_7_adj']]

# ── 2.  TEC / PSN / SI monthly data ─────────────────────────────────────────
def load_scinda(tec_fp, psn_fp, si_fp) -> pd.DataFrame:
    """Merge TEC, PSN, SI on 1-minute timestamps and clean."""
    print("Reading individual files...")
    
    # Read TEC data (1 observation per minute)
    tec = pd.read_csv(tec_fp, sep=r'\t')
    print(f"TEC shape: {tec.shape}")
    print(f"TEC columns: {list(tec.columns)}")
    
    # Read PSN data (1 observation per minute)
    psn = pd.read_csv(psn_fp, sep=r'\t')
    print(f"PSN shape: {psn.shape}")
    print(f"PSN columns: {list(psn.columns)}")
    
    # Read SI data (10 observations per minute)
    si = pd.read_csv(si_fp, sep=r'\t')
    print(f"SI shape before averaging: {si.shape}")
    print(f"SI columns: {list(si.columns)}")

    # FIXED: Flexible timestamp parsing for all datasets
    for d, name in [(tec, 'TEC'), (psn, 'PSN'), (si, 'SI')]:
        try:
            d['datetime'] = pd.to_datetime(d['time.stamp'], format='%d-%m-%Y %H:%M')
        except ValueError:
            try:
                d['datetime'] = pd.to_datetime(d['time.stamp'], format='%Y-%m-%d %H:%M:%S')
            except ValueError:
                try:
                    d['datetime'] = pd.to_datetime(d['time.stamp'], format='%Y-%m-%d %H:%M')
                except ValueError:
                    d['datetime'] = pd.to_datetime(d['time.stamp'], infer_datetime_format=True)
        print(f"{name} datetime range: {d['datetime'].min()} to {d['datetime'].max()}")

    # NEW: Average SI data to 1-minute resolution
    print("Averaging SI data from 10 observations per minute to 1 per minute...")
    
    # Round SI datetime to minute for grouping
    si['datetime_minute'] = si['datetime'].dt.floor('T')
    
    # Average numerical columns, take mode for categorical (PRN)
    si_avg = si.groupby('datetime_minute').agg({
        'az': 'mean',
        'el': 'mean',
        'L1S4': 'mean',
        'ROTI': 'mean',
        'PRN': lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
    }).reset_index()
    
    si_avg.rename(columns={'datetime_minute': 'datetime'}, inplace=True)
    print(f"SI shape after averaging: {si_avg.shape}")

    # FIXED: Dynamic column detection for TEC data
    print("Detecting TEC column names...")
    tec_columns = list(tec.columns)
    
    # Find TEC columns with flexible matching
    tec_raw_col = None
    tec_cal_col = None
    
    for col in tec_columns:
        if 'TECR' in col and 'calibrated' not in col:
            tec_raw_col = col
        elif 'TECR' in col and 'calibrated' in col:
            tec_cal_col = col
        elif 'TEC' in col and 'calibrated' in col:
            tec_cal_col = col
    
    print(f"Found TEC raw column: {tec_raw_col}")
    print(f"Found TEC calibrated column: {tec_cal_col}")
    
    if not tec_cal_col:
        raise ValueError(f"Could not find calibrated TEC column in: {tec_columns}")

    # Merge datasets
    print("Merging datasets...")
    
    # Start with TEC data - use detected column names
    tec_cols_to_use = ['datetime']
    if tec_raw_col:
        tec_cols_to_use.append(tec_raw_col)
    if tec_cal_col:
        tec_cols_to_use.append(tec_cal_col)
    
    merged = tec[tec_cols_to_use].copy()
    
    # Merge PSN data
    psn_cols = ['datetime', 'mean.x', 'mean.y', 'mean.z', 'sigma.x', 'sigma.y', 'sigma.z']
    merged = merged.merge(psn[psn_cols], on='datetime', how='inner')
    
    # Merge averaged SI data
    si_cols = ['datetime', 'az', 'el', 'L1S4', 'ROTI', 'PRN']
    merged = merged.merge(si_avg[si_cols], on='datetime', how='inner')
    
    print(f"Merged dataset shape: {merged.shape}")

    # FIXED: Dynamic column renaming
    rename_dict = {
        'mean.x': 'lat', 
        'mean.y': 'lon', 
        'mean.z': 'alt',
        'L1S4': 'S4'
    }
    
    # Add TEC column renaming
    if tec_cal_col:
        rename_dict[tec_cal_col] = 'TEC'
    
    merged.rename(columns=rename_dict, inplace=True)

    # Quality filtering
    print("Applying quality filters...")
    initial_rows = len(merged)
    
    # Filter out invalid coordinates and low elevation
    merged = merged[
        (merged['sigma.x'].notna()) & (merged['sigma.x'] != 0) &
        (merged['sigma.y'].notna()) & (merged['sigma.y'] != 0) &
        (merged['sigma.z'].notna()) & (merged['sigma.z'] != 0) &
        (merged['el'] >= 15)
    ]
    
    print(f"Filtered out {initial_rows - len(merged)} rows with invalid data")
    print(f"Final SCINDA dataset shape: {merged.shape}")

    # Drop unnecessary columns
    cols_to_drop = ['sigma.x', 'sigma.y', 'sigma.z']
    if tec_raw_col and tec_raw_col in merged.columns:
        cols_to_drop.append(tec_raw_col)
    
    # Only drop columns that exist
    cols_to_drop = [col for col in cols_to_drop if col in merged.columns]
    if cols_to_drop:
        merged.drop(columns=cols_to_drop, inplace=True)

    # Create date/time columns
    merged['date'] = merged['datetime'].dt.strftime('%d-%m-%Y')
    merged['time'] = merged['datetime'].dt.strftime('%H:%M')
    
    print(f"Final columns after processing: {list(merged.columns)}")
    return merged

# ── 3.  Nearest-time merge (GFZ → SCINDA) ───────────────────────────────────
def attach_gfz(scinda: pd.DataFrame, gfz: pd.DataFrame) -> pd.DataFrame:
    """For each SCINDA row pick GFZ record with nearest datetime."""
    scinda = scinda.copy()
    gfz_sorted = gfz.sort_values('datetime')
    scinda['datetime'] = pd.to_datetime(scinda['date'] + ' ' + scinda['time'],
                                        format='%d-%m-%Y %H:%M')
    final = pd.merge_asof(
        scinda.sort_values('datetime'),
        gfz_sorted,
        on='datetime',
        direction='nearest'
    )
    return final.drop(columns='datetime')

# ── 4.  Main routine ────────────────────────────────────────────────────────
def main():
    # ── edit these four paths ─────────────────────────────────────────
    gfz_file = Path('D:\IIT T dataset\sample-ghz.txt')
    tec_file = Path('D:\IIT T dataset\SCN_TEC_1min_2018_m03.dat')
    psn_file = Path('D:\IIT T dataset\eqTime_PSN_data_2018_m03.dat')
    si_file  = Path('D:\IIT T dataset\eqTime_SI_SCN_data_2018_m03.dat')
    # ────────────────────────────────────────────────────────────────────

    print("=" * 60)
    print("IONOSPHERIC TEC PREDICTION DATA PROCESSING")
    print("=" * 60)
    
    print("\n1. Loading GFZ solar/geomagnetic data...")
    gfz = load_gfz(gfz_file)
    print(f"✅ GFZ data loaded: {len(gfz)} records")
    
    print("\n2. Loading and processing SCINDA data...")
    scinda = load_scinda(tec_file, psn_file, si_file)
    print(f"✅ SCINDA data processed: {len(scinda)} records")
    
    print("\n3. Merging with GFZ data...")
    final_df = attach_gfz(scinda, gfz)
    print(f"✅ Final merged dataset: {len(final_df)} records")

    # FIXED: Dynamic column selection
    available_columns = list(final_df.columns)
    desired_columns = [
        'date', 'time', 'TEC',           # Target and temporal
        'lat', 'lon', 'alt',             # Spatial
        'az', 'el', 'S4', 'ROTI', 'PRN', # Satellite/scintillation
        'Kp', 'ap', 'Ap', 'SN', 'F10_7_obs'  # Solar/geomagnetic
    ]
    
    # Only keep columns that actually exist
    final_columns = [col for col in desired_columns if col in available_columns]
    print(f"Available columns: {available_columns}")
    print(f"Keeping columns: {final_columns}")
    
    final_df = final_df[final_columns]

    # Save to CSV
    output_file = 'final_merged_dataset.csv'
    final_df.to_csv(output_file, index=False)
    
    print(f"\n✅ Dataset saved to: {output_file}")
    print(f" Final dataset shape: {final_df.shape}")
    print(f" Columns: {list(final_df.columns)}")
    
    # Display sample statistics
    if 'TEC' in final_df.columns:
        print("\n📈 Sample Statistics:")
        print(f"   • TEC range: {final_df['TEC'].min():.2f} - {final_df['TEC'].max():.2f} TECU")
        print(f"   • Latitude range: {final_df['lat'].min():.2f}° - {final_df['lat'].max():.2f}°")
        print(f"   • Kp range: {final_df['Kp'].min():.1f} - {final_df['Kp'].max():.1f}")
        print(f"   • F10.7 range: {final_df['F10_7_obs'].min():.1f} - {final_df['F10_7_obs'].max():.1f} s.f.u.")
    
    print("\n Dataset ready for ML training!")

if __name__ == '__main__':
    main()

<>:194: SyntaxWarning: invalid escape sequence '\I'
<>:195: SyntaxWarning: invalid escape sequence '\I'
<>:196: SyntaxWarning: invalid escape sequence '\I'
<>:197: SyntaxWarning: invalid escape sequence '\I'
<>:194: SyntaxWarning: invalid escape sequence '\I'
<>:195: SyntaxWarning: invalid escape sequence '\I'
<>:196: SyntaxWarning: invalid escape sequence '\I'
<>:197: SyntaxWarning: invalid escape sequence '\I'
C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:194: SyntaxWarning: invalid escape sequence '\I'
  gfz_file = Path('D:\IIT T dataset\sample-ghz.txt')
C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:195: SyntaxWarning: invalid escape sequence '\I'
  tec_file = Path('D:\IIT T dataset\SCN_TEC_1min_2018_m03.dat')
C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:196: SyntaxWarning: invalid escape sequence '\I'
  psn_file = Path('D:\IIT T dataset\eqTime_PSN_data_2018_m03.dat')
C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:197: 

IONOSPHERIC TEC PREDICTION DATA PROCESSING

1. Loading GFZ solar/geomagnetic data...
✅ GFZ data loaded: 248 records

2. Loading and processing SCINDA data...
Reading individual files...


C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:40: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tec = pd.read_csv(tec_fp, sep=r'\t')


TEC shape: (44640, 3)
TEC columns: ['time.stamp', 'TECR', 'TECR.calibrated']


C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:45: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  psn = pd.read_csv(psn_fp, sep=r'\t')


PSN shape: (44640, 14)
PSN columns: ['time.stamp', 'year', 'month', 'day', 'sec.f.midnight', 'h', 'm', 's', 'mean.x', 'mean.y', 'mean.z', 'sigma.x', 'sigma.y', 'sigma.z']


C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:50: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  si = pd.read_csv(si_fp, sep=r'\t')


SI shape before averaging: (256043, 6)
SI columns: ['time.stamp', 'az', 'el', 'L1S4', 'ROTI', 'PRN']
TEC datetime range: 2018-03-01 00:00:00 to 2018-03-31 23:59:00
PSN datetime range: 2018-03-01 00:00:00 to 2018-03-31 23:59:00
SI datetime range: 2018-03-01 00:00:00 to 2018-03-31 23:59:00
Averaging SI data from 10 observations per minute to 1 per minute...


C:\Users\vvard\AppData\Local\Temp\ipykernel_12432\74403314.py:72: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  si['datetime_minute'] = si['datetime'].dt.floor('T')


SI shape after averaging: (44640, 6)
Detecting TEC column names...
Found TEC raw column: TECR
Found TEC calibrated column: TECR.calibrated
Merging datasets...
Merged dataset shape: (44640, 14)
Applying quality filters...
Filtered out 16111 rows with invalid data
Final SCINDA dataset shape: (28529, 14)
Final columns after processing: ['datetime', 'TEC', 'lat', 'lon', 'alt', 'az', 'el', 'S4', 'ROTI', 'PRN', 'date', 'time']
✅ SCINDA data processed: 28529 records

3. Merging with GFZ data...
✅ Final merged dataset: 28529 records
Available columns: ['TEC', 'lat', 'lon', 'alt', 'az', 'el', 'S4', 'ROTI', 'PRN', 'date', 'time', 'Kp', 'ap', 'Ap', 'SN', 'F10_7_obs', 'F10_7_adj']
Keeping columns: ['date', 'time', 'TEC', 'lat', 'lon', 'alt', 'az', 'el', 'S4', 'ROTI', 'PRN', 'Kp', 'ap', 'Ap', 'SN', 'F10_7_obs']

✅ Dataset saved to: final_merged_dataset.csv
 Final dataset shape: (28529, 16)
 Columns: ['date', 'time', 'TEC', 'lat', 'lon', 'alt', 'az', 'el', 'S4', 'ROTI', 'PRN', 'Kp', 'ap', 'Ap', 'SN'

In [1]:
import pandas as pd


file_path = r'D:\IIT T dataset\final_merged_dataset.csv'  # Using raw string

df = pd.read_csv(file_path)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
df

,date,time,TEC,lat,lon,alt,az,el,S4,ROTI,PRN,Kp,ap,Ap,SN,F10_7_obs
0,01-03-2018,00:00,9.15,38.779316,-9.139703,128.123200,205.837500,36.762500,0.072500,9.515000,1.0,0.667,3,6,12,67.2
1,01-03-2018,00:01,9.14,38.779323,-9.139702,126.733350,206.025000,36.812500,0.086250,9.847500,1.0,0.667,3,6,12,67.2
2,01-03-2018,00:02,9.12,38.779321,-9.139703,127.373833,206.175000,36.837500,0.081250,9.778750,1.0,0.667,3,6,12,67.2
3,01-03-2018,00:03,9.12,38.779319,-9.139709,127.070050,195.366667,33.888889,0.098889,9.242222,1.0,0.667,3,6,12,67.2
4,01-03-2018,00:04,9.11,38.779324,-9.139710,127.703383,195.500000,33.944444,0.095556,10.596667,1.0,0.667,3,6,12,67.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28524,31-03-2018,23:55,7.76,38.779312,-9.139703,128.412717,185.900000,50.040000,0.069000,9.615556,1.0,0.667,3,6,13,69.0
28525,31-03-2018,23:56,7.76,38.779313,-9.139702,128.672133,186.220000,50.040000,0.070000,9.813333,1.0,0.667,3,6,13,69.0
28526,31-03-2018,23:57,7.77,38.779312,-9.139702,128.079183,186.590000,50.070000,0.072000,9.947778,1.0,0.667,3,6,13,69.0
28527,31-03-2018,23:58,7.29,38.779315,-9.139700,128.655283,186.960000,50.100000,0.063000,9.964444,1.0,0.667,3,6,13,69.0
